##1: Load & Prepare Data (Gold Layer)

In [0]:
# Check your current setup
print("Current catalog:", spark.sql("SELECT current_catalog()").collect()[0][0])
print("Current database:", spark.sql("SELECT current_database()").collect()[0][0])

# List available catalogs
print("\nAvailable catalogs:")
spark.sql("SHOW CATALOGS").show()

# List databases in current catalog
current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
spark.sql(f"SHOW DATABASES IN {current_catalog}").show()

Current catalog: workspace
Current database: default

Available catalogs:
+--------------+
|       catalog|
+--------------+
|ecommerce_prod|
|    ml_catalog|
|       my_shop|
|       samples|
|        system|
|     workspace|
+--------------+

+------------------+
|      databaseName|
+------------------+
|           default|
|         ecommerce|
|information_schema|
+------------------+



In [0]:

print("1. LOADING & PREPARING DATA (Gold Layer)")

# Switch to ecommerce_prod catalog 
spark.sql("USE CATALOG ecommerce_prod")
print("✓ Switched to catalog: ecommerce_prod")

# Load October and November data
oct_df = spark.read.option("header", True).option("inferSchema", True)\
    .csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

nov_df = spark.read.option("header", True).option("inferSchema", True)\
    .csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

# Combine datasets
raw_df = oct_df.unionByName(nov_df)

print(f"✓ October data: {oct_df.count():,} rows")
print(f"✓ November data: {nov_df.count():,} rows")
print(f"✓ Combined data: {raw_df.count():,} rows")

# Create temp view for Genie
raw_df.createOrReplaceTempView("ecommerce_events")
print("✓ Created temp view: ecommerce_events")

# Create gold schema 
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
print("✓ Created/verified schema: gold")

# Save to Gold layer for persistent access
raw_df.write.mode("overwrite").saveAsTable("gold.ecommerce_events")
print("✓ Saved to: ecommerce_prod.gold.ecommerce_events")


1. LOADING & PREPARING DATA (Gold Layer)
✓ Switched to catalog: ecommerce_prod
✓ October data: 42,448,764 rows
✓ November data: 67,501,979 rows
✓ Combined data: 109,950,743 rows
✓ Created temp view: ecommerce_events
✓ Created/verified schema: gold
✓ Saved to: ecommerce_prod.gold.ecommerce_events


##2: Genie - Natural Language to SQL Queries

In [0]:
print("\n2. GENIE: NATURAL LANGUAGE TO SQL QUERIES")
print("="*60)
print("To use Genie, follow these steps in Databricks SQL:")
print("1. Go to: SQL → Genie (in left sidebar)")
print("2. Select your table: ecommerce_prod.gold.ecommerce_events")
print("3. Ask questions in natural language")
print("="*60)

# Show example Genie queries
genie_examples = [
    "Show me total revenue by category",
    "Which products have the highest conversion rate?",
    "What's the trend of daily purchases over time?",
    "Find customers who viewed but never purchased",
    "Show top 10 products by revenue",
    "What is the average purchase value by brand?"
]

print("\nExample Genie Queries to Try:")
for i, query in enumerate(genie_examples, 1):
    print(f"{i}. \"{query}\"")


2. GENIE: NATURAL LANGUAGE TO SQL QUERIES
To use Genie, follow these steps in Databricks SQL:
1. Go to: SQL → Genie (in left sidebar)
2. Select your table: gold.ecommerce_events
3. Ask questions in natural language

Example Genie Queries to Try:
1. "Show me total revenue by category"
2. "Which products have the highest conversion rate?"
3. "What's the trend of daily purchases over time?"
4. "Find customers who viewed but never purchased"
5. "Show top 10 products by revenue"
6. "What is the average purchase value by brand?"


##3: Manual Implementation of Genie Queries (for reference)

In [0]:
print("\n3. MANUAL SQL IMPLEMENTATION (What Genie Generates)")

print("\nQuery 1: Total Revenue by Category")
revenue_by_category = spark.sql("""
    SELECT 
        category_code,
        SUM(price) AS total_revenue
    FROM gold.ecommerce_events
    WHERE category_code IS NOT NULL 
        AND price IS NOT NULL
        AND event_type = 'purchase'
    GROUP BY category_code
    ORDER BY total_revenue DESC
    LIMIT 10
""")
print("Top 10 Categories by Revenue:")
revenue_by_category.show(truncate=False)

print("\nQuery 2: Products with Highest Conversion Rate")
conversion_rates = spark.sql("""
    WITH product_stats AS (
        SELECT 
            product_id,
            COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
            COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases
        FROM gold.ecommerce_events
        WHERE product_id IS NOT NULL
        GROUP BY product_id
    )
    SELECT 
        product_id,
        views,
        purchases,
        ROUND(purchases * 100.0 / NULLIF(views, 0), 2) AS conversion_rate_percent
    FROM product_stats
    WHERE views > 10  -- Filter out products with few views
    ORDER BY conversion_rate_percent DESC
    LIMIT 10
""")
print("Top 10 Products by Conversion Rate (min 10 views):")
conversion_rates.show(truncate=False)

print("\nQuery 3: Customers Who Viewed But Never Purchased")
view_only_customers = spark.sql("""
    SELECT DISTINCT 
        user_id
    FROM gold.ecommerce_events
    WHERE event_type = 'view'
        AND user_id IS NOT NULL
        AND user_id NOT IN (
            SELECT DISTINCT user_id
            FROM gold.ecommerce_events
            WHERE event_type = 'purchase'
                AND user_id IS NOT NULL
        )
    LIMIT 10
""")
print("Sample of Customers Who Viewed But Never Purchased:")
view_only_customers.show(truncate=False)
print(f"Total count: {view_only_customers.count():,}")


3. MANUAL SQL IMPLEMENTATION (What Genie Generates)

Query 1: Total Revenue by Category
Top 10 Categories by Revenue:
+--------------------------------+--------------------+
|category_code                   |total_revenue       |
+--------------------------------+--------------------+
|electronics.smartphone          |3.3487128498000944E8|
|electronics.video.tv            |2.0880559020000063E7|
|computers.notebook              |1.9658316959999967E7|
|electronics.clocks              |1.1371042719999947E7|
|appliances.kitchen.washer       |1.0460552919999974E7|
|electronics.audio.headphone     |9208629.479999918   |
|appliances.kitchen.refrigerators|8552734.310000002   |
|appliances.environment.vacuum   |4478737.029999995   |
|electronics.tablet              |3131226.7899999986  |
|computers.desktop               |2673459.679999999   |
+--------------------------------+--------------------+


Query 2: Products with Highest Conversion Rate
Top 10 Products by Conversion Rate (min 10 views

####4: Mosaic AI - Sentiment Analysis (NLP Example)

In [0]:
print("\n4. MOSAIC AI: SENTIMENT ANALYSIS (NLP Example)")

try:
    import transformers
    
    from transformers import pipeline
    import torch
    
    print("✓ Transformers library available")
    
    # Load sentiment analysis model
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    print(f"Loading model: {model_name}")
    
    classifier = pipeline(
        task="sentiment-analysis",
        model=model_name,
        device=0 if torch.cuda.is_available() else -1
    )
    
    # Sample product reviews for analysis
    reviews = [
        "This product is amazing, I love it!",
        "Very poor quality, totally disappointed",
        "Average product, not bad but not great either",
        "Excellent value for money, highly recommended",
        "Broke after 2 days, worst purchase ever"
    ]
    
    # Run sentiment analysis
    print("\nRunning sentiment analysis on sample reviews...")
    results = classifier(reviews)
    
    # Display results
    print("\nSentiment Analysis Results:")
    print("-" * 60)
    for review, result in zip(reviews, results):
        sentiment = result['label']
        confidence = result['score']
        print(f"Review: {review[:50]}...")
        print(f"  → Sentiment: {sentiment} (Confidence: {confidence:.4f})")
        print()
    
    # Calculate summary statistics
    positive_count = sum(1 for r in results if r['label'] == 'POSITIVE')
    negative_count = len(results) - positive_count
    positive_ratio = positive_count / len(results)
    
    print(f"Summary: {positive_count}/{len(results)} positive ({positive_ratio:.1%})")
    
except ImportError:
    print("⚠️ Transformers library not available. Installing...")
    %pip install transformers torch
    print("Please restart the notebook or run the cell again.")
    results = None
    reviews = []



4. MOSAIC AI: SENTIMENT ANALYSIS (NLP Example)
✓ Transformers library available
Loading model: distilbert-base-uncased-finetuned-sst-2-english


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu



Running sentiment analysis on sample reviews...

Sentiment Analysis Results:
------------------------------------------------------------
Review: This product is amazing, I love it!...
  → Sentiment: POSITIVE (Confidence: 0.9999)

Review: Very poor quality, totally disappointed...
  → Sentiment: NEGATIVE (Confidence: 0.9998)

Review: Average product, not bad but not great either...
  → Sentiment: NEGATIVE (Confidence: 0.9949)

Review: Excellent value for money, highly recommended...
  → Sentiment: POSITIVE (Confidence: 0.9999)

Review: Broke after 2 days, worst purchase ever...
  → Sentiment: NEGATIVE (Confidence: 0.9996)

Summary: 2/5 positive (40.0%)


## 5: Log NLP Model to MLflow


In [0]:
print("\n5. LOGGING NLP MODEL TO MLFLOW")

import mlflow
import pandas as pd

if results:
    try:
        # Create MLflow experiment for NLP
        mlflow.set_experiment("/Shared/Day14_AI_Analytics")
        print("✓ MLflow experiment set: /Shared/Day14_AI_Analytics")
        
        with mlflow.start_run(run_name="sentiment_analysis_distilbert"):
            # Log parameters
            mlflow.log_param("model_name", model_name)
            mlflow.log_param("task", "sentiment-analysis")
            mlflow.log_param("sample_size", len(reviews))
            
            # Log metrics
            mlflow.log_metric("positive_ratio", positive_ratio)
            mlflow.log_metric("negative_ratio", 1 - positive_ratio)
            mlflow.log_metric("avg_confidence", 
                             sum(r['score'] for r in results) / len(results))
            
            # Log example predictions as artifact
            results_dict = {
                "reviews": reviews,
                "predictions": [
                    {"label": r['label'], "score": float(r['score'])} 
                    for r in results
                ]
            }
            mlflow.log_dict(results_dict, "sentiment_results.json")
            
            print("✓ Sentiment analysis logged to MLflow")
            
            # Create insights DataFrame
            insights_df = pd.DataFrame(results)
            insights_df.insert(0, 'review', reviews)
            print("\nAI-Powered Business Insights:")
            print("-" * 60)
            print(insights_df.to_string(index=False))
            
    except Exception as e:
        print(f"⚠️ MLflow logging failed: {e}")
else:
    print("⚠️ Skipping MLflow logging (no sentiment results available)")

2026/01/22 13:26:46 INFO mlflow.tracking.fluent: Experiment with name '/Shared/Day14_AI_Analytics' does not exist. Creating a new experiment.



5. LOGGING NLP MODEL TO MLFLOW
✓ MLflow experiment set: /Shared/Day14_AI_Analytics
✓ Sentiment analysis logged to MLflow

AI-Powered Business Insights:
------------------------------------------------------------
                                       review    label    score
          This product is amazing, I love it! POSITIVE 0.999886
      Very poor quality, totally disappointed NEGATIVE 0.999817
Average product, not bad but not great either NEGATIVE 0.994931
Excellent value for money, highly recommended POSITIVE 0.999851
      Broke after 2 days, worst purchase ever NEGATIVE 0.999572


##6: AI-Powered Business Insights

In [0]:
print("\n6. AI-POWERED BUSINESS INSIGHTS")

# Generate additional insights from the data
print("\nGenerating AI-powered insights from e-commerce data...")

# Insight 1: Purchase trends by day of week
day_of_week_insight = spark.sql("""
    SELECT 
        date_format(event_time, 'EEEE') as day_of_week,
        COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) as purchase_count,
        AVG(CASE WHEN event_type = 'purchase' THEN price END) as avg_purchase_value
    FROM gold.ecommerce_events
    WHERE event_type = 'purchase'
    GROUP BY date_format(event_time, 'EEEE'), dayofweek(event_time)
    ORDER BY dayofweek(event_time)
""")

print("\nInsight 1: Purchase Trends by Day of Week")
day_of_week_insight.show()

# Insight 2: Product category analysis
category_insight = spark.sql("""
    SELECT 
        category_code,
        COUNT(DISTINCT user_id) as unique_customers,
        SUM(CASE WHEN event_type = 'purchase' THEN price END) as total_revenue,
        COUNT(CASE WHEN event_type = 'view' THEN 1 END) as total_views,
        COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) as total_purchases,
        ROUND(
            COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) * 100.0 / 
            COUNT(CASE WHEN event_type = 'view' THEN 1 END), 
            2
        ) as conversion_rate_percent
    FROM gold.ecommerce_events
    WHERE category_code IS NOT NULL
    GROUP BY category_code
    ORDER BY total_revenue DESC
    LIMIT 5
""")

print("\nInsight 2: Top 5 Categories Performance")
category_insight.show(truncate=False)


6. AI-POWERED BUSINESS INSIGHTS

Generating AI-powered insights from e-commerce data...

Insight 1: Purchase Trends by Day of Week
+-----------+--------------+------------------+
|day_of_week|purchase_count|avg_purchase_value|
+-----------+--------------+------------------+
|     Sunday|        353614| 303.8484599874588|
|     Monday|        201246| 301.8847979090253|
|    Tuesday|        211303| 304.3687469179293|
|  Wednesday|        216416| 302.7826587682961|
|   Thursday|        210222|304.43161548267744|
|     Friday|        207510| 303.8981994120716|
|   Saturday|        259477| 308.5168849647625|
+-----------+--------------+------------------+


Insight 2: Top 5 Categories Performance
+-------------------------+----------------+--------------------+-----------+---------------+-----------------------+
|category_code            |unique_customers|total_revenue       |total_views|total_purchases|conversion_rate_percent|
+-------------------------+----------------+------------------

In [0]:
print("=== SIMULATING GENIE NATURAL LANGUAGE QUERIES ===\n")

# Dictionary of questions and their SQL equivalents
genie_queries = {
    "Show me total revenue by category": """
        SELECT 
            category_code,
            SUM(price) AS total_revenue
        FROM ecommerce_prod.gold.ecommerce_events
        WHERE event_type = 'purchase'
        GROUP BY category_code
        ORDER BY total_revenue DESC
        LIMIT 10
    """,
    
    "Which products have the highest conversion rate?": """
        WITH product_stats AS (
            SELECT 
                product_id,
                COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS views,
                COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS purchases
            FROM ecommerce_prod.gold.ecommerce_events
            WHERE product_id IS NOT NULL
            GROUP BY product_id
        )
        SELECT 
            product_id,
            views,
            purchases,
            ROUND(purchases * 100.0 / NULLIF(views, 0), 2) AS conversion_rate_percent
        FROM product_stats
        WHERE views > 0
        ORDER BY conversion_rate_percent DESC
        LIMIT 10
    """,
    
    "Find customers who viewed but never purchased": """
        SELECT DISTINCT 
            user_id
        FROM ecommerce_prod.gold.ecommerce_events
        WHERE event_type = 'view'
            AND user_id IS NOT NULL
            AND user_id NOT IN (
                SELECT DISTINCT user_id
                FROM ecommerce_prod.gold.ecommerce_events
                WHERE event_type = 'purchase'
                    AND user_id IS NOT NULL
            )
        LIMIT 10
    """,
    
    "What's the trend of daily purchases over time?": """
        SELECT 
            DATE(event_time) AS purchase_date,
            COUNT(*) AS daily_purchases,
            SUM(price) AS daily_revenue
        FROM ecommerce_prod.gold.ecommerce_events
        WHERE event_type = 'purchase'
        GROUP BY DATE(event_time)
        ORDER BY purchase_date
        LIMIT 10
    """
}

# Simulate Genie interaction
for question, sql_query in genie_queries.items():
    print(f"\n👤 User asks: '{question}'")
    print("🤖 Genie generates this SQL:")
    print(sql_query)
    
    # Execute and show results
    result = spark.sql(sql_query)
    print(f"📊 Results ({result.count()} rows):")
    result.show(5, truncate=False)
    print("-" * 80)

=== SIMULATING GENIE NATURAL LANGUAGE QUERIES ===


👤 User asks: 'Show me total revenue by category'
🤖 Genie generates this SQL:

        SELECT 
            category_code,
            SUM(price) AS total_revenue
        FROM ecommerce_prod.gold.ecommerce_events
        WHERE event_type = 'purchase'
        GROUP BY category_code
        ORDER BY total_revenue DESC
        LIMIT 10
    
📊 Results (10 rows):
+----------------------+--------------------+
|category_code         |total_revenue       |
+----------------------+--------------------+
|electronics.smartphone|3.3487128498000944E8|
|NULL                  |5.280544381000084E7 |
|electronics.video.tv  |2.0880559020000063E7|
|computers.notebook    |1.9658316959999967E7|
|electronics.clocks    |1.1371042719999947E7|
+----------------------+--------------------+
only showing top 5 rows
--------------------------------------------------------------------------------

👤 User asks: 'Which products have the highest conversion rate?'
🤖 Gen

In [0]:
print("\n=== SIMULATING MOSAIC AI PLAYGROUND ===\n")

# Simulate text summarization
print("👤 User request: 'Summarize the key trends in our e-commerce data'")
print("🤖 Mosaic AI generates:")

# Analyze and "summarize" based on actual data
summary_queries = [
    "SELECT COUNT(DISTINCT user_id) as total_customers FROM ecommerce_prod.gold.ecommerce_events",
    "SELECT COUNT(*) as total_purchases, SUM(price) as total_revenue FROM ecommerce_prod.gold.ecommerce_events WHERE event_type = 'purchase'",
    "SELECT AVG(price) as avg_purchase_value FROM ecommerce_prod.gold.ecommerce_events WHERE event_type = 'purchase'"
]

summary_results = {}
for query in summary_queries:
    result = spark.sql(query).collect()[0]
    key = list(result.asDict().keys())[0]
    summary_results[key] = result[key]

print(f"""
📈 E-commerce Data Summary:
• Total Customers: {summary_results.get('total_customers', 0):,}
• Total Purchases: {summary_results.get('total_purchases', 0):,}
• Total Revenue: ${summary_results.get('total_revenue', 0):,.2f}
• Average Purchase Value: ${summary_results.get('avg_purchase_value', 0):.2f}

Key Insights:
1. Electronics (smartphones) drive majority of revenue
2. Conversion rates vary significantly by product category  
3. Opportunities in customer retention (view → purchase conversion)
""")


=== SIMULATING MOSAIC AI PLAYGROUND ===

👤 User request: 'Summarize the key trends in our e-commerce data'
🤖 Mosaic AI generates:

📈 E-commerce Data Summary:
• Total Customers: 5,316,649
• Total Purchases: 1,659,788
• Total Revenue: $0.00
• Average Purchase Value: $304.35

Key Insights:
1. Electronics (smartphones) drive majority of revenue
2. Conversion rates vary significantly by product category  
3. Opportunities in customer retention (view → purchase conversion)

